In [1]:
# Import dependencies and initialize findspark
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install hvplot

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

from pyspark import SparkFiles

import pandas as pd
import tensorflow as tf
import h5py

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

In [3]:
# Read in data from Github
from pyspark import SparkFiles
import requests

url = "https://raw.githubusercontent.com/fraserclaire/Group3_Project4/main/Resources/energy_cancer_statePerYear.csv"
response = requests.get(url)

with open("energy_cancer_statePerYear.csv", "wb") as f:
    f.write(response.content)


spark.sparkContext.addFile(url)
energy_cancer_df = spark.read.csv(SparkFiles.get('energy_cancer_statePerYear.csv'), inferSchema=True, header=True)

# Show DataFrame
energy_cancer_df.show()



+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------------------+-----------------+-------------+-----------------+-------------+-----------------+-------------+-----------------+-----

In [4]:
# Read in datasets from github
url = "<insert_pathtocsv_here>.csv"
spark.sparkContext.addFile(url)
cancer_model = spark.read.csv(SparkFiles.get("<insert_csvname_here>.csv"), sep=",", header=True)

Py4JJavaError: An error occurred while calling o32.addFile.
: java.io.FileNotFoundException: File file:/content/<insert_pathtocsv_here>.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1755)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1733)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Metho

In [ ]:
# Create a view for cancer dataset
cancer_model.createOrReplaceTempView('cancer')
spark.sql("""SELECT * FROM  cancer""").show()

In [17]:
# Read in datasets from github
url3 = "https://raw.githubusercontent.com/fraserclaire/Group3_Project4/main/Resources/all_weather_all_energy.csv"
spark.sparkContext.addFile(url3)
weather_model = spark.read.csv(SparkFiles.get("all_weather_all_energy.csv"), sep=",", header=True)

# Create a view for weather dataset
weather_model.createOrReplaceTempView('weather')
spark.sql("""SELECT * FROM  weather""").show(121)

+---+------------------+------------------+-------+----------+------+------------------+-------+-----+----+----------+------------+------------+---------------+------------+-------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [25]:
sql = """
  SELECT
  YEAR AS YEAR,
  State,
  (SUM(`2011(C)`)/12) AS `2011_Coal_Consumption`
  FROM weather
  WHERE YEAR = 2011
  AND State = 'Alabama'
  GROUP BY YEAR, State
  """

spark.sql(sql).show()

+----+-------+---------------------+
|YEAR|  State|2011_Coal_Consumption|
+----+-------+---------------------+
|2011|Alabama|             651032.0|
+----+-------+---------------------+



In [6]:
# Convert the PySpark DataFrame to a pandas DataFrame
weather_model_pandas = weather_model.toPandas()
weather_model_pandas.head()

,_c0,latitude,longitude,State,State Abbr,region,sub-region,country,month,year,...,2011(W),2012(W),2013(W),2014(W),2015(W),2016(W),2017(W),2018(W),2019(W),2020(W)
0,0,32.794376373291016,-86.80929565429688,Alabama,AL,South,East South Central,US,1,2011,...,0,0,0,0,0,0,0,0,0,0
1,1,32.794376373291016,-86.80929565429688,Alabama,AL,South,East South Central,US,2,2011,...,0,0,0,0,0,0,0,0,0,0
2,2,32.794376373291016,-86.80929565429688,Alabama,AL,South,East South Central,US,3,2011,...,0,0,0,0,0,0,0,0,0,0
3,3,32.794376373291016,-86.80929565429688,Alabama,AL,South,East South Central,US,4,2011,...,0,0,0,0,0,0,0,0,0,0
4,4,32.794376373291016,-86.80929565429688,Alabama,AL,South,East South Central,US,5,2011,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Look at State value counts for binning
weather_model_pandas_state_counts = weather_model_pandas['State'].value_counts()
weather_model_pandas_state_counts

State
Alabama                 120
Pennsylvania            120
Nevada                  120
New Hampshire           120
New Jersey              120
New Mexico              120
New York                120
North Carolina          120
North Dakota            120
Ohio                    120
Oklahoma                120
Oregon                  120
Rhode Island            120
Montana                 120
South Carolina          120
South Dakota            120
Tennessee               120
Texas                   120
Utah                    120
Vermont                 120
Virginia                120
Washington              120
West Virginia           120
Wisconsin               120
Nebraska                120
Missouri                120
Alaska                  120
Idaho                   120
Arizona                 120
Arkansas                120
California              120
Colorado                120
Connecticut             120
Delaware                120
District of Columbia    120
Florida       

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
weather_energy_dummies = pd.get_dummies(weather_model_pandas, dtype=int)
weather_energy_dummies

,_c0_0,_c0_1,_c0_10,_c0_100,_c0_1000,_c0_1001,_c0_1002,_c0_1003,_c0_1004,_c0_1005,...,2020(W)_5192,2020(W)_55123,2020(W)_5647,2020(W)_59047,2020(W)_63334,2020(W)_7037,2020(W)_76950,2020(W)_79914,2020(W)_810430,2020(W)_81231
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
weather_energy_dummies['2011(C)_1695239']

0       0
1       0
2       0
3       0
4       0
       ..
6115    0
6116    0
6117    0
6118    0
6119    0
Name: 2011(C)_1695239, Length: 6120, dtype: int64

In [10]:
# Split our preprocessed data into our features and target arrays
y = weather_energy_dummies["2011(C)_1695239"].values
X = weather_energy_dummies.drop(["2011(C)_1695239"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                347880    
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 347941 (1.33 MB)
Trainable params: 347941 (1.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
144/144 [==============================] - 3s 16ms/step - loss: 0.3435 - accuracy: 0.9196
Epoch 2/100
144/144 [==============================] - 1s 10ms/step - loss: 0.1569 - accuracy: 1.0000
Epoch 3/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0748 - accuracy: 0.9998
Epoch 4/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0505 - accuracy: 1.0000
Epoch 5/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0402 - accuracy: 1.0000
Epoch 6/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0383 - accuracy: 1.0000
Epoch 7/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 8/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0346 - accuracy: 1.0000
Epoch 9/100
144/144 [==============================] - 2s 15ms/step - loss: 0.0328 - accuracy: 1.0000
Epoch 10/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0311 -

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

48/48 - 0s - loss: 2.4997e-04 - accuracy: 1.0000 - 402ms/epoch - 8ms/step
Loss: 0.0002499688125681132, Accuracy: 1.0
